This notebook is to demonstrate the interface of the developed environment

Replace the variable *PATH_TO_ROOT* to run the notebook

In [1]:
import sys

PATH_TO_ROOT = 'C:/Users/walter/Desktop/git/AlphaBuilding-ResCommunity'
sys.path.insert(0,PATH_TO_ROOT)

In [3]:
import pandas as pd
import numpy as np

from gym_AlphaBuilding.envs import residential
from bin.util.distribution import utility
from bin.util.weather import noaa_weather

In [4]:
city = 'Berkeley'
state = 'CA'
start_date = '2018-07-01'
final_date = '2018-08-01'
year = 2018

In [5]:
ttc = utility.get_ttc(state)
teq = utility.get_teq(state)
tsp, trange = utility.get_comfort_temp('cooling', 'ASHRAE PMV')

In [6]:
SAMPLE_SIZE = 100
STEP_SIZE = 15 # min
sim_horizon = (start_date, final_date)

In [7]:
# Download weather data from NOAA weather stations
# If you want to use your own weather data, skip this step
address = '{}, {}'.format(city, state)
station_ids, station_names = noaa_weather.find_closest_weather_station(noaa_weather.geocode_address(address))

# You might need to try a couple of weather stations, because some weather stations have large missing rate 
weather = noaa_weather.download_weather(station_ids[0], year)
weather = weather.tz_convert('America/Los_Angeles').tz_localize(None)  # remove tz-awareness

# truncate and resample the weather data to fit the simulation horizon and time step
weather_h = weather.resample('1H').mean()  # hourlu average to remove noise
# ambient_temp = weather_h.resample('{}min'.format(STEP_SIZE)).interpolate()[['Temperature']]
weather_h = weather_h.resample('{}min'.format(STEP_SIZE)).interpolate()
weather_h = weather_h.truncate(*sim_horizon)

In [8]:
weather_h = weather_h.resample('{}min'.format(STEP_SIZE)).interpolate()

In [9]:
env = residential.AlphaResEnv(sample_size  = SAMPLE_SIZE, 
                           step_size    = STEP_SIZE, 
                           sim_horizon  = (start_date, final_date),
                           ambient_weather = weather_h, 
                           ttc = ttc, 
                           teq = teq, 
                           tsp = tsp, 
                           trange = trange,
                           hvac_mode = 'cooling only',
                           otherHG_method = 'Ecobee',
                           noise_sigma = 0.01,
                           cost_weight = (10, 1))

C:\Users\walter\Anaconda3\envs\ecobee_env\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float16
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [10]:
env.Q_h.mean()

17.017112179094926

In [11]:
len(env.T_high)

100

In [12]:
env.reset()

array([14.4       ,  4.        ,  0.        , 24.76672991, 26.46570434,
       24.83045372, 24.40266909, 25.9825167 , 25.51985327, 25.67811294,
       26.59943781, 26.40984396, 25.29229681, 23.98095905, 25.38339704,
       26.76669106, 24.67781783, 25.28526032, 24.23280562, 25.93229342,
       25.28941238, 24.10537832, 26.31856719, 26.49482239, 24.7933331 ,
       26.38038274, 24.77820504, 26.18150736, 24.71457652, 25.50329354,
       25.56519041, 25.86906003, 24.52195075, 25.28036047, 25.39131303,
       24.69122963, 25.18503879, 25.29571005, 25.48774973, 24.86653458,
       25.19761857, 24.60728128, 25.92479041, 26.83831166, 26.77038832,
       26.82392258, 25.60614929, 25.2279601 , 24.38574759, 26.06385446,
       26.3036843 , 26.1211748 , 24.87676596, 25.58052389, 25.12811416,
       25.92482288, 24.02087952, 25.88228966, 26.2016718 , 24.27735184,
       25.02500388, 26.22513952, 25.71158195, 25.19286197, 24.2551101 ,
       26.36831501, 25.80855803, 25.87372035, 25.56026628, 26.74

In [13]:
env.obs[env.obs_names.index('hour')]

0.0

In [14]:
# 1 for heating, 2 for cooling, 0 for free floating
env.step(np.ones(100)*2)

(array([14.275     ,  4.        ,  0.        , 22.82207882, 26.10039196,
        23.17113666, 23.81616538, 23.68476062, 25.19552023, 25.7859952 ,
        24.87914877, 25.03914401, 23.73247044, 24.01152676, 23.52391552,
        26.76860418, 23.84257774, 24.8295625 , 21.59263787, 25.48250071,
        23.60972716, 23.04744486, 24.03833568, 25.2048401 , 23.70410843,
        25.01108319, 23.25228158, 25.48253782, 24.18080134, 25.04236144,
        23.44493558, 24.9087481 , 22.71958328, 24.40636841, 24.35858322,
        24.96149545, 24.40716628, 24.59185881, 24.61002925, 25.27194789,
        24.46527433, 22.94926896, 25.44239346, 25.84620504, 24.20753513,
        24.98745874, 24.66239779, 25.20266731, 23.5503187 , 25.85329801,
        24.92514274, 23.48456793, 23.73157551, 24.87032009, 23.26423684,
        25.83030567, 24.01478631, 25.63545255, 25.31938692, 22.71174941,
        24.67165449, 25.22791902, 23.64415779, 24.52579308, 23.39469395,
        26.2063099 , 24.00475454, 23.31272396, 24.5